In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

def clean_and_prepare_data(df):
    """
    Clean the data by handling infinities and extreme values
    """
    df_clean = df.copy()
    
    # Replace infinities with NaN
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    
    # Handle numerical columns
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        # Calculate statistics excluding NaN
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        
        # Cap extreme values
        df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)
        
        # Fill remaining NaN with median
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
    return df_clean

def prepare_time_series_data(df, target_col, sequence_length=10):
    """
    Prepare data for time series models with proper cleaning and scaling
    """
    # Clean the data first
    df_clean = clean_and_prepare_data(df)
    
    # Separate continuous and binary features
    continuous_features = ['passenger_count', 'current_speed', 'distance_to_next_stop', 
                         'eta_minutes', 'speed_distance_ratio']
    
    binary_features = ['weather_condition_foggy', 'weather_condition_rainy', 
                      'weather_condition_storm', 'weather_condition_sunny',
                      'weather_condition_windy', 'traffic_condition_heavy',
                      'traffic_condition_jam', 'traffic_condition_light',
                      'traffic_condition_normal', 'hour_sin', 'hour_cos']
    
    # Scale continuous features
    scaler = RobustScaler()
    df_scaled = df_clean.copy()
    df_scaled[continuous_features] = scaler.fit_transform(df_clean[continuous_features])
    
    # Combine all features
    features = continuous_features + binary_features
    data = df_scaled[features].astype('float32').values
    
    sequences = []
    targets = []
    
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:(i + sequence_length)])
        targets.append(data[i + sequence_length][0])
    
    return np.array(sequences), np.array(targets), scaler

def create_lstm_model(sequence_length, n_features):
    """
    Create LSTM model with improved architecture
    """
    model = Sequential([
        LSTM(128, activation='relu', input_shape=(sequence_length, n_features), return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        
        LSTM(64, activation='relu', return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        
        LSTM(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='huber', metrics=['mae'])
    return model

def create_cnn_model(sequence_length, n_features):
    """
    Create CNN model with improved architecture
    """
    model = Sequential([
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same',
               input_shape=(sequence_length, n_features)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        Dropout(0.3),
        
        Flatten(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='huber', metrics=['mae'])
    return model

def train_sarima(data, seasonal_period=24):
    """
    Train SARIMA model with automatic parameter selection
    """
    print("Training SARIMA model...")
    best_aic = float('inf')
    best_order = None
    best_seasonal_order = None
    
    # Grid search for best parameters
    for p in range(2):
        for d in range(2):
            for q in range(2):
                for P in range(2):
                    for D in range(2):
                        for Q in range(2):
                            try:
                                model = SARIMAX(data,
                                              order=(p,d,q),
                                              seasonal_order=(P,D,Q,seasonal_period),
                                              enforce_stationarity=False,
                                              enforce_invertibility=False)
                                results = model.fit(disp=False)
                                if results.aic < best_aic:
                                    best_aic = results.aic
                                    best_order = (p,d,q)
                                    best_seasonal_order = (P,D,Q,seasonal_period)
                            except:
                                continue
    
    print(f"Best SARIMA parameters - Order: {best_order}, Seasonal Order: {best_seasonal_order}")
    
    final_model = SARIMAX(data,
                         order=best_order,
                         seasonal_order=best_seasonal_order,
                         enforce_stationarity=False,
                         enforce_invertibility=False)
    return final_model.fit(disp=False)

def plot_training_history(history, model_name):
    """
    Plot training history for deep learning models
    """
    plt.figure(figsize=(12, 4))
    
    plt.subplot(121)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} - Training History (Loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(122)
    plt.plot(history.history['mae'], label='Training MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.title(f'{model_name} - Training History (MAE)')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def plot_predictions(y_true, y_pred, model_name, scaler=None):
    """
    Plot actual vs predicted values
    """
    if scaler is not None:
        # Create dummy array for inverse transform
        dummy = np.zeros((len(y_true), scaler.n_features_in_))
        dummy[:, 0] = y_true  # Assuming passenger_count is first feature
        y_true_orig = scaler.inverse_transform(dummy)[:, 0]
        
        dummy[:, 0] = y_pred.flatten()
        y_pred_orig = scaler.inverse_transform(dummy)[:, 0]
    else:
        y_true_orig = y_true
        y_pred_orig = y_pred
    
    plt.figure(figsize=(12, 6))
    plt.plot(y_true_orig, label='Actual', alpha=0.7)
    plt.plot(y_pred_orig, label=f'{model_name} Predictions', alpha=0.7)
    plt.title(f'{model_name} - Actual vs Predicted')
    plt.xlabel('Time Steps')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.show()

def main():
    # Load data
    print("Loading data...")
    df = pd.read_csv('preprocessed_bus_data.csv')
    
    # Print initial data info
    print("\nInitial data info:")
    print(df.info())
    print("\nChecking for infinities:")
    print(df.isin([np.inf, -np.inf]).sum())
    
    # Prepare sequences
    print("\nPreparing sequences...")
    sequence_length = 10
    X, y, scaler = prepare_time_series_data(df, 'passenger_count', sequence_length)
    
    print("\nSequence shapes:")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    lr_schedule = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )
    
    callbacks = [early_stopping, lr_schedule]
    
    # Train LSTM
    print("\nTraining LSTM model...")
    lstm_model = create_lstm_model(sequence_length, X.shape[2])
    lstm_history = lstm_model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )
    
    # Train CNN
    print("\nTraining CNN model...")
    cnn_model = create_cnn_model(sequence_length, X.shape[2])
    cnn_history = cnn_model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )
    
    # Train SARIMA
    print("\nPreparing SARIMA data...")
    sarima_data = df['passenger_count']
    train_size = int(len(sarima_data) * 0.8)
    train, test = sarima_data[:train_size], sarima_data[train_size:]
    
    print("Training SARIMA model...")
    sarima_model = train_sarima(train)
    
    # Make predictions
    print("\nMaking predictions...")
    lstm_pred = lstm_model.predict(X_test)
    cnn_pred = cnn_model.predict(X_test)
    sarima_pred = sarima_model.forecast(len(test))
    
    # Plot results
    print("\nPlotting results...")
    plot_training_history(lstm_history, 'LSTM')
    plot_training_history(cnn_history, 'CNN')
    
    plot_predictions(y_test, lstm_pred, 'LSTM', scaler)
    plot_predictions(y_test, cnn_pred, 'CNN', scaler)
    plot_predictions(test, sarima_pred, 'SARIMA')
    
    # Print evaluation metrics
    print("\nModel Evaluation:")
    print("LSTM - MSE:", mean_squared_error(y_test, lstm_pred))
    print("CNN - MSE:", mean_squared_error(y_test, cnn_pred))
    print("SARIMA - MSE:", mean_squared_error(test, sarima_pred))
    
    return lstm_model, cnn_model, sarima_model

if __name__ == "__main__":
    try:
        lstm_model, cnn_model, sarima_model = main()
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Loading data...

Initial data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 39 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   timestamp                  100000 non-null  object 
 1   bus_id                     100000 non-null  object 
 2   route_id                   100000 non-null  object 
 3   stop_sequence              100000 non-null  int64  
 4   current_stop_name          100000 non-null  object 
 5   next_stop_name             100000 non-null  object 
 6   day_of_week                100000 non-null  int64  
 7   is_holiday                 100000 non-null  bool   
 8   is_peak_hour               100000 non-null  bool   
 9   passenger_count            100000 non-null  float64
 10  current_speed              100000 non-null  float64
 11  distance_to_next_stop      100000 non-null  float64
 12  current_lat                100000 non-null  float64